In [28]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Global plot settings
plt.rcParams['figure.dpi'] = 150   # Set the resolution of the plots to 150 DPI
plt.style.use('fivethirtyeight')

Importing the Sales Data (Updated by Lina)

In [29]:
sales = pd.read_csv('/workspaces/bakery_prediction/0_DataPreparation/Processed/featured_data_categorised.csv', encoding= 'utf-8')

In [30]:
sales.head()

,id,Datum,Warengruppe,Umsatz,Bewoelkung,Temperatur,Windgeschwindigkeit,Wettercode,KielerWoche,Woche,Monat,Wochentag,Feiertag,Jahreszeit,Ferien
0,1307011,2013-07-01,1,148.828353,6.0,17.8375,15.0,cloudy,0,27,7,1,0,3,0
1,1307021,2013-07-02,1,159.793757,3.0,17.3125,10.0,cloudy,0,27,7,2,0,3,0
2,1307031,2013-07-03,1,111.885594,7.0,21.0750,6.0,rainy,0,27,7,3,0,3,0
3,1307041,2013-07-04,1,168.864941,7.0,18.8500,7.0,cloudy,0,27,7,4,0,3,0
4,1307051,2013-07-05,1,171.280754,5.0,19.9750,12.0,cloudy,0,27,7,5,0,3,0


In [31]:
# creating a work copy

sales_wc = sales.copy()

# converting date column to datetime type
sales_wc["Datum"] = pd.to_datetime(sales["Datum"], errors="coerce", format="%Y-%m-%d")


# other time intelligence columns on the sales Table
sales_wc["Year"] = sales_wc["Datum"].dt.year                   # year column
sales_wc["Month_Num"] = sales_wc["Datum"].dt.month                 # month column (number)
sales_wc["Month_Name"] = sales_wc["Datum"].dt.month_name()           # month name
sales_wc["Weekday_Num"] = sales_wc["Datum"].dt.weekday               # weekday Code
sales_wc["Weekday"] = sales_wc["Datum"].dt.day_name()              # Weekday Name 

In [32]:
sales_wc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9334 entries, 0 to 9333
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   id                   9334 non-null   int64         
 1   Datum                9334 non-null   datetime64[ns]
 2   Warengruppe          9334 non-null   int64         
 3   Umsatz               9334 non-null   float64       
 4   Bewoelkung           9334 non-null   float64       
 5   Temperatur           9334 non-null   float64       
 6   Windgeschwindigkeit  9334 non-null   float64       
 7   Wettercode           9334 non-null   object        
 8   KielerWoche          9334 non-null   int64         
 9   Woche                9334 non-null   int64         
 10  Monat                9334 non-null   int64         
 11  Wochentag            9334 non-null   int64         
 12  Feiertag             9334 non-null   int64         
 13  Jahreszeit           9334 non-nul

In [33]:
products = {
   "Warengruppe": [1, 2, 3, 4, 5,6],
   "Product_Name": ["Bread (Brot)", "Roles (Brötchen)", "Croissant", "Pastry (Konditorei)", "Cakes (Kuchen)","Seasonal Products (Saison Brot)"],
}

product_table = pd.DataFrame(products)

In [34]:
sales_wc = sales_wc.merge(product_table, on='Warengruppe')

sales_wc.head()

,id,Datum,Warengruppe,Umsatz,Bewoelkung,Temperatur,Windgeschwindigkeit,Wettercode,KielerWoche,Woche,...,Wochentag,Feiertag,Jahreszeit,Ferien,Year,Month_Num,Month_Name,Weekday_Num,Weekday,Product_Name
0,1307011,2013-07-01,1,148.828353,6.0,17.8375,15.0,cloudy,0,27,...,1,0,3,0,2013,7,July,0,Monday,Bread (Brot)
1,1307021,2013-07-02,1,159.793757,3.0,17.3125,10.0,cloudy,0,27,...,2,0,3,0,2013,7,July,1,Tuesday,Bread (Brot)
2,1307031,2013-07-03,1,111.885594,7.0,21.0750,6.0,rainy,0,27,...,3,0,3,0,2013,7,July,2,Wednesday,Bread (Brot)
3,1307041,2013-07-04,1,168.864941,7.0,18.8500,7.0,cloudy,0,27,...,4,0,3,0,2013,7,July,3,Thursday,Bread (Brot)
4,1307051,2013-07-05,1,171.280754,5.0,19.9750,12.0,cloudy,0,27,...,5,0,3,0,2013,7,July,4,Friday,Bread (Brot)


In [35]:
sales_wc.to_csv('/workspaces/bakery_prediction/0_DataPreparation/Processed/featured_data_pd_names.csv', index=False)

## Descriptive Analytics